### Extract top-N scoring clips per Site ID and SN. 

Then we will sort the predictions by score using the column 'Confidence' and select the top-N clip per site and survey night. Then save this selection (including the column begin Path, confidence and offset) to use it in the next step that will be extract the clips from long recorings using Audio from OpenSoundscape.

In [1]:
import os
import pandas as pd
from pathlib import Path
from opensoundscape import Audio
from tqdm.autonotebook import tqdm

/home/agosjuncosa/anaconda3/envs/opensoundscape/lib/python3.9/site-packages/opensoundscape/ml/cnn.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [2]:
main_dir = "/mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/"
top_n = 1  # Set the number of top clips per Site_ID and SN
clip_duration = 3  # seconds

In [3]:
# STEP 1: Load and rename columns
predictions = pd.read_csv(os.path.join(main_dir, "fepowl_SelectionTable.txt"), sep='\t')

predictions.rename(columns={
    'Begin Path': 'file_path',
    'File Offset (s)': 'offset',
    'Common Name': 'class',
    'Species Code': 'class_code',
    'Confidence': 'score',
}, inplace=True)

Sort predictions by score and Site ID, SN & Score and select the top N clips 

In [4]:
# STEP 2: Group and select top-N clips per Site_ID and SN

predictions_sorted = predictions.sort_values(by='score', ascending=False)
predictions_sorted.head()

,Selection,class,class_code,score,file_path,offset,Site_ID,SN
59632,59633,Ferruginous Pygmy-Owl,fepowl,0.9999,/mnt/d/Disco3_Backup/night_recordings/CH13-ARU...,21.0,CH13,13
127523,127524,Ferruginous Pygmy-Owl,fepowl,0.9999,/mnt/d/Disco3_Backup/night_recordings/M19-ARU2...,105.0,M19,10
59641,59642,Ferruginous Pygmy-Owl,fepowl,0.9999,/mnt/d/Disco3_Backup/night_recordings/CH13-ARU...,51.0,CH13,13
59633,59634,Ferruginous Pygmy-Owl,fepowl,0.9999,/mnt/d/Disco3_Backup/night_recordings/CH13-ARU...,24.0,CH13,13
59638,59639,Ferruginous Pygmy-Owl,fepowl,0.9999,/mnt/d/Disco3_Backup/night_recordings/CH13-ARU...,42.0,CH13,13


In [5]:
top_clips = predictions_sorted.groupby(['Site_ID', 'SN']).head(top_n).reset_index(drop=True)
top_clips.head()

,Selection,class,class_code,score,file_path,offset,Site_ID,SN
0,59633,Ferruginous Pygmy-Owl,fepowl,0.9999,/mnt/d/Disco3_Backup/night_recordings/CH13-ARU...,21.0,CH13,13
1,127524,Ferruginous Pygmy-Owl,fepowl,0.9999,/mnt/d/Disco3_Backup/night_recordings/M19-ARU2...,105.0,M19,10
2,128409,Ferruginous Pygmy-Owl,fepowl,0.9998,/mnt/d/Disco3_Backup/night_recordings/M19-ARU2...,57.0,M19,11
3,272387,Ferruginous Pygmy-Owl,fepowl,0.9998,/mnt/d/Disco4_Backup/night_recordings/M37-ARU3...,9.0,M37,24
4,32656,Ferruginous Pygmy-Owl,fepowl,0.9998,/mnt/d/Disco2_Backup/night_recordings/M1-ARU7/...,33.0,M1,15


Using the 'file_path' column & 'offset', **extract the clips from long recordings for validation**

In [6]:
# STEP 3: Extract and save clips
for idx, row in top_clips.iterrows():
    site_id = row['Site_ID']
    sn = row['SN']
    offset = int(row['offset'])
    score = row['score']
    full_audio_path = Path(row['file_path'])
    
    # Construct subdirectory and filename
    out_dir = Path(main_dir)
    #site_dir.mkdir(parents=True, exist_ok=True)

    # Format output filename
    output_filename = f"{score:.4f}_{offset}_{site_id}_SN{sn}.WAV".lower() 
    output_path = out_dir / output_filename
    
    try:
        # Load and extract clip
        audio = Audio.from_file(full_audio_path, offset=offset, duration=clip_duration)
        audio.save(output_path)
        print(f"Saved: {output_path}")
    except Exception as e:
        print(f"Error processing {full_audio_path} at offset {offset}: {e}")

print(f"\n Done! Clips saved to: {main_dir}")

Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.9999_21_ch13_sn13.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.9999_105_m19_sn10.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.9998_57_m19_sn11.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.9998_9_m37_sn24.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.9998_33_m1_sn15.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.9998_42_m37_sn21.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.9998_78_m37_sn20.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.9998_12_m37_sn25.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.9998_90_m37_sn14.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.9997_75_m37_sn27.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.9996_60_m37_sn26.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.9

/home/agosjuncosa/anaconda3/envs/opensoundscape/lib/python3.9/site-packages/opensoundscape/audio.py:340: UserWarning: Audio object is shorter than requested duration: 2.0 sec instead of 3 sec
  warnings.warn(error_msg)


Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0051_72_ch45_sn18.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0050_24_m39_sn6.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0050_42_ch41_sn12.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0050_48_m44_sn6.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0050_111_ch25_sn2.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0050_72_p39_sn4.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0050_54_p21_sn6.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0050_48_p30_sn21.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0050_60_p30_sn15.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0049_84_m44_sn5.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0049_12_ch43_sn7.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.004

/home/agosjuncosa/anaconda3/envs/opensoundscape/lib/python3.9/site-packages/opensoundscape/audio.py:340: UserWarning: Audio object is shorter than requested duration: 2.0 sec instead of 3 sec
  warnings.warn(error_msg)


Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0018_48_p22_sn8.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0018_21_p6_sn8.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0018_9_m30_sn6.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0018_27_m39_sn10.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0018_39_m28_sn31.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0018_93_p28_sn3.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0018_30_p26_sn11.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0018_51_m45_sn16.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0018_99_p40_sn14.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0018_42_p41_sn18.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0018_105_ch7_sn12.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0018_2

/home/agosjuncosa/anaconda3/envs/opensoundscape/lib/python3.9/site-packages/opensoundscape/audio.py:340: UserWarning: Audio object is shorter than requested duration: 1.70425 sec instead of 3 sec
  warnings.warn(error_msg)


Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_12_p1_sn2.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_102_m29_sn21.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_81_p36_sn12.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_63_p37_sn4.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_63_m34_sn33.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_39_m39_sn14.wav


/home/agosjuncosa/anaconda3/envs/opensoundscape/lib/python3.9/site-packages/opensoundscape/audio.py:340: UserWarning: Audio object is shorter than requested duration: 2.637583333333333 sec instead of 3 sec
  warnings.warn(error_msg)


Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_96_ch45_sn2.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_27_ch26_sn6.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_0_ch26_sn15.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_96_p27_sn8.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_21_p26_sn5.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_57_m24_sn12.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_93_m20_sn18.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_66_m17_sn18.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0011_63_p42_sn3.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0010_78_p14_sn10.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0010_48_p13_sn18.wav
Saved: /mnt/d/night_recordings_analysis/Ferruginous Pygmy-Owl/0.0010